In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:

# * Etape 1 : Load and inspect the dataset.


# Définir le chemin des données
path = "../data_final_project/KuaiRec 2.0/data/"

# Charger les données
small_matrix = pd.read_csv(path + "small_matrix.csv")
big_matrix = pd.read_csv(path + "big_matrix.csv")
item_categories = pd.read_csv(path + "item_categories.csv")
item_features = pd.read_csv(path + "item_daily_features.csv")
social_network = pd.read_csv(path + "social_network.csv")
user_features = pd.read_csv(path + "user_features.csv")

try:
    # Method 1: Using Python engine instead of C engine
    captions = pd.read_csv(path + "kuairec_caption_category.csv", engine='python')
    print("Captions loaded successfully with Python engine")
except Exception as e:
    print(f"Method 1 failed: {e}")
    try:
        # Method 2: Increase buffer size and skip bad lines
        captions = pd.read_csv(path + "kuairec_caption_category.csv", 
                              engine='python', 
                              on_bad_lines='skip',
                              encoding='utf-8')
        print("Captions loaded successfully with skipping bad lines")
    except Exception as e:
        print(f"Method 2 failed: {e}")
        print("Unable to load captions file. Proceeding without it.")
        captions = None
        

Captions loaded successfully with Python engine


In [3]:

# * Etape 2 :  Traitement des valeurs manquantes et inconsistantes

def clear_matrix(matrix):
    """
    Fonction pour nettoyer la matrice en traitant les valeurs manquantes et les valeurs extrêmes.
    
    Args:
        matrix (DataFrame): La matrice d'interactions à nettoyer
        
    Returns:
        DataFrame: La matrice nettoyée
    """
    # Traiter les valeurs manquantes
    if matrix['timestamp'].isnull().sum() > 0:
        # Supprimer les lignes avec timestamp manquant
        matrix_cleaned = matrix.dropna(subset=['timestamp'])
        print(f"\nAprès suppression des valeurs manquantes, la matrice a {matrix_cleaned.shape[0]} lignes")
    else:
        matrix_cleaned = matrix.copy()
    
    # Traitement des valeurs extrêmes dans watch_ratio

    #q_low representes la valeur max du 1% le plus bas
    q_low = matrix_cleaned['watch_ratio'].quantile(0.05) # ? jouer avec ce parametre
    print(f"Valeur max du 1% le plus bas: {q_low}")
    #q_high representes la valeur min du 1% le plus haut
    q_high = matrix_cleaned['watch_ratio'].quantile(0.9999) # ? jouer avec ce parametre
    print(f"Valeur min du 1% le plus haut: {q_high}")

    # Filtrer les valeurs extrêmes
    matrix_filtered = matrix_cleaned[(matrix_cleaned['watch_ratio'] >= q_low) & 
                                    (matrix_cleaned['watch_ratio'] <= q_high)]
    print(f"Après filtrage des valeurs extrêmes, la matrice a {matrix_filtered.shape[0]} lignes")
    
    return matrix_filtered
small_matrix_cleaned = clear_matrix(small_matrix)


Après suppression des valeurs manquantes, la matrice a 4494578 lignes
Valeur max du 1% le plus bas: 0.10910294790311445
Valeur min du 1% le plus haut: 56.26230695693068
Après filtrage des valeurs extrêmes, la matrice a 4269399 lignes


In [4]:
# Fonction pour convertir la string en liste d'entiers
def parse_item_categories_feat(cat_str):
    """
    Convertit la représentation textuelle des catégories en liste d'entiers
    """
    try:
        if isinstance(cat_str, str):
            # Supprimer les crochets et convertir en liste d'entiers
            return [int(x) for x in cat_str.strip('[]').split(',') if x.strip()]
        print(f"Erreur de type pour la chaîne de catégories: {cat_str}")
        return []
    except Exception as e:
        print(f"Erreur lors du parsing de la chaîne de catégories: {cat_str}, erreur: {e}")
        return []

# Appliquer la fonction à la colonne 'feat' en créant une nouvelle colonne 'category_list'
item_categories['category_list'] = item_categories['feat'].apply(parse_item_categories_feat)
# Créer un dictionnaire de mapping video_id -> catégories pour un accès efficace
video_to_categories = dict(zip(item_categories['video_id'], item_categories['category_list']))

In [5]:
def add_categories_to_interactions(interactions_df):
    """
    Ajoute les catégories de vidéos aux données d'interaction
    """
    # Créer une colonne de catégories en utilisant le mapping
    interactions_df['categories'] = interactions_df['video_id'].map(video_to_categories)
    
    # Remplacer les valeurs NaN par des listes vides
    interactions_df['categories'] = interactions_df['categories'].fillna('').apply(lambda x: [] if x == '' else x)
    
    # Calculer le nombre de catégories par vidéo
    interactions_df['category_count'] = interactions_df['categories'].apply(len)
    
    return interactions_df

# Fusionner les métadonnées des catégories avec le jeu de données small_matrix nettoyé
small_matrix_with_categories = add_categories_to_interactions(small_matrix_cleaned)

# Vérifier la fusion des données
print("\nExemple d'interactions avec catégories:")
print(small_matrix_with_categories[['user_id', 'video_id', 'categories', 'category_count']].head())
print(f"\nNombre de vidéos sans catégorie: {sum(small_matrix_with_categories['category_count'] == 0)}")


Exemple d'interactions avec catégories:
   user_id  video_id    categories  category_count
0       14       148  [11, 28, 19]               3
1       14       183          [28]               1
2       14      3649           [9]               1
3       14      5262          [25]               1
4       14      8234           [6]               1

Nombre de vidéos sans catégorie: 0


In [6]:
# Solution alternative - nettoyer puis convertir en entiers si possible
if captions is not None:
    try:
        # Fonction pour extraire seulement les chiffres
        def extract_digits(value):
            import re
            digits = re.findall(r'\d+', str(value))
            return int(digits[0]) if digits else None
        
        # Appliquer la fonction
        captions['video_id_int'] = captions['video_id'].apply(extract_digits)
        
        # Filtrer les lignes où la conversion a fonctionné
        captions_valid = captions.dropna(subset=['video_id_int'])
        
        # Créer le dictionnaire avec les ID numériques
        video_captions = dict(zip(captions_valid['video_id_int'], captions_valid['caption']))
        
        # Ajouter le texte des captions
        small_matrix_with_categories['caption'] = small_matrix_with_categories['video_id'].map(video_captions)

        print("\nExemple d'interactions avec captions:")
        print(small_matrix_with_categories[['user_id', 'video_id', 'categories', 'caption']].head())
    
    except Exception as e:
        print(f"Erreur lors de l'ajout des captions: {e}")
        print("Poursuite du traitement sans les données de captions.")


Exemple d'interactions avec captions:
   user_id  video_id    categories  \
0       14       148  [11, 28, 19]   
1       14       183          [28]   
2       14      3649           [9]   
3       14      5262          [25]   
4       14      8234           [6]   

                                             caption  
0  美60岁奶奶与21少年一见钟情，兴奋分享初次体验，称：升华了感情！ @推广小助手(O4030...  
1          合肥高铁南站，他带了瓶开过封的茅台被拦下，酒值4000元他舍不得扔，就一饮而尽。   
2                     美女裤兜上插“菜刀”吓坏旁人   哦豁，拔出来才发现是手机壳  
3    生了个洋娃娃，婆婆非要发出来给大家看看，看看有没有人点赞，一分钟催我看一下，一分钟催我看一下😂  
4  #郝劭文 小时候火遍全国，跟释小龙一起成为一代人最喜爱的童星，长大后却没能大火，你还记得那个...  


In [7]:
# Vérifier si item_features est disponible
if 'item_features' in locals() and not item_features.empty:
    # Convertir date en datetime si nécessaire
    if not pd.api.types.is_datetime64_any_dtype(item_features['date']):
        item_features['date'] = pd.to_datetime(item_features['date'], format='%Y%m%d', errors='coerce')
    
    # Sélectionner les colonnes d'intérêt
    selected_features = ['video_id', 'date', 'play_progress', 
                         'like_cnt', 'comment_cnt', 'share_cnt']
    
    # Créer des agrégations pertinentes pour des compteurs quotidiens
    video_features_agg = item_features[selected_features].groupby('video_id').agg({
        'date': 'max',                    # Date la plus récente pour référence
        'play_progress': 'mean',          # Progression moyenne sur toute la période
        'like_cnt': 'sum',                # Total des likes sur toute la période
        'comment_cnt': 'sum',             # Total des commentaires sur toute la période
        'share_cnt': 'sum'                # Total des partages sur toute la période
    }).reset_index()
    
    # Calculer le nombre de jours avec des données pour chaque vidéo
    days_per_video = item_features.groupby('video_id')['date'].nunique()
    video_features_agg['days_tracked'] = video_features_agg['video_id'].map(days_per_video)
    
    # Calculer les moyennes quotidiennes
    video_features_agg['avg_daily_likes'] = video_features_agg['like_cnt'] / video_features_agg['days_tracked']
    video_features_agg['avg_daily_comments'] = video_features_agg['comment_cnt'] / video_features_agg['days_tracked']
    video_features_agg['avg_daily_shares'] = video_features_agg['share_cnt'] / video_features_agg['days_tracked']
    
    # Renommer les colonnes pour refléter l'agrégation
    video_features_agg = video_features_agg.rename(columns={
        'play_progress': 'avg_play_progress',
        'like_cnt': 'total_likes',
        'comment_cnt': 'total_comments',
        'share_cnt': 'total_shares'
    })
    
    # Fusionner avec notre dataset
    small_matrix_with_metadata = pd.merge(
        small_matrix_with_categories,
        video_features_agg,
        on='video_id',
        how='left',
        suffixes=('', '_features')
    )
    
    # Afficher un exemple du résultat
    print("\nExemple d'interactions avec métadonnées agrégées:")
    print(small_matrix_with_metadata[['user_id', 'video_id', 'categories', 
                                     'total_likes', 'avg_daily_likes', 
                                     'total_comments', 'avg_daily_comments']].head())
else:
    small_matrix_with_metadata = small_matrix_with_categories
# suppression de la colonne 'date' et 'timestamp' car elles ne sont plus nécessaires
small_matrix_with_metadata.drop(columns=['date', 'timestamp'], inplace=True, errors='ignore')


Exemple d'interactions avec métadonnées agrégées:
   user_id  video_id    categories  total_likes  avg_daily_likes  \
0       14       148  [11, 28, 19]        31199       495.222222   
1       14       183          [28]       210670      3343.968254   
2       14      3649           [9]       279456      4435.809524   
3       14      5262          [25]      1363469     21642.365079   
4       14      8234           [6]        87997      1396.777778   

   total_comments  avg_daily_comments  
0            4574           72.603175  
1            5684           90.222222  
2            5900           93.650794  
3           29188          463.301587  
4            4328           68.698413  


In [8]:

# * Récapitulatif des données fusionnées
print(f"\nDimensions finales du dataset fusionné: {small_matrix_with_metadata.shape}")
print(f"Nombre de colonnes après fusion: {len(small_matrix_with_metadata.columns)}")
print("Colonnes disponibles après fusion:")
print(small_matrix_with_metadata.columns.tolist())


Dimensions finales du dataset fusionné: (4269399, 18)
Nombre de colonnes après fusion: 18
Colonnes disponibles après fusion:
['user_id', 'video_id', 'play_duration', 'video_duration', 'time', 'watch_ratio', 'categories', 'category_count', 'caption', 'date_features', 'avg_play_progress', 'total_likes', 'total_comments', 'total_shares', 'days_tracked', 'avg_daily_likes', 'avg_daily_comments', 'avg_daily_shares']


In [9]:
print(small_matrix_with_metadata)

import joblib

# Sauvegarde
joblib.dump(small_matrix_with_metadata, 'small_matrix_with_metadata.joblib')

         user_id  video_id  play_duration  video_duration  \
0             14       148           4381            6067   
1             14       183          11635            6100   
2             14      3649          22422           10867   
3             14      5262           4479            7908   
4             14      8234           4602           11000   
...          ...       ...            ...             ...   
4269394     7162      9177           5315           37205   
4269395     7162      4987          10085            8167   
4269396     7162      7988          50523           49319   
4269397     7162      6533           2190            8000   
4269398     7162      6523          11909            7255   

                            time  watch_ratio    categories  category_count  \
0        2020-07-05 05:27:48.378     0.722103  [11, 28, 19]               3   
1        2020-07-05 05:28:00.057     1.907377          [28]               1   
2        2020-07-05 05:29:09.4

['small_matrix_with_metadata.joblib']